# Desafio 5

Neste desafio, vamos praticar sobre redução de dimensionalidade com PCA e seleção de variáveis com RFE. Utilizaremos o _data set_ [Fifa 2019](https://www.kaggle.com/karangadiya/fifa19), contendo originalmente 89 variáveis de mais de 18 mil jogadores do _game_ FIFA 2019.

> Obs.: Por favor, não modifique o nome das funções de resposta.

## _Setup_ geral

In [43]:
from math import sqrt

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as sct
import seaborn as sns
import statsmodels.api as sm
import statsmodels.stats as st
from sklearn.decomposition import PCA
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression

from loguru import logger

In [2]:
# Algumas configurações para o matplotlib.
# %matplotlib inline

from IPython.core.pylabtools import figsize


figsize(8,6)

sns.set()

In [3]:
fifa = pd.read_csv("fifa.csv")

In [4]:
columns_to_drop = ["Unnamed: 0", "ID", "Name", "Photo", "Nationality", "Flag",
                   "Club", "Club Logo", "Value", "Wage", "Special", "Preferred Foot",
                   "International Reputation", "Weak Foot", "Skill Moves", "Work Rate",
                   "Body Type", "Real Face", "Position", "Jersey Number", "Joined",
                   "Loaned From", "Contract Valid Until", "Height", "Weight", "LS",
                   "ST", "RS", "LW", "LF", "CF", "RF", "RW", "LAM", "CAM", "RAM", "LM",
                   "LCM", "CM", "RCM", "RM", "LWB", "LDM", "CDM", "RDM", "RWB", "LB", "LCB",
                   "CB", "RCB", "RB", "Release Clause"
]

try:
    fifa.drop(columns_to_drop, axis=1, inplace=True)
except KeyError:
    logger.warning(f"Columns already dropped")

## Inicia sua análise a partir daqui

In [39]:
# Sua análise começa aqui.
fifa.head()

,Age,Overall,Potential,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,...,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes
0,31,94,94,84.0,95.0,70.0,90.0,86.0,97.0,93.0,...,75.0,96.0,33.0,28.0,26.0,6.0,11.0,15.0,14.0,8.0
1,33,94,94,84.0,94.0,89.0,81.0,87.0,88.0,81.0,...,85.0,95.0,28.0,31.0,23.0,7.0,11.0,15.0,14.0,11.0
2,26,92,93,79.0,87.0,62.0,84.0,84.0,96.0,88.0,...,81.0,94.0,27.0,24.0,33.0,9.0,9.0,15.0,15.0,11.0
3,27,91,93,17.0,13.0,21.0,50.0,13.0,18.0,21.0,...,40.0,68.0,15.0,21.0,13.0,90.0,85.0,87.0,88.0,94.0
4,27,91,92,93.0,82.0,55.0,92.0,82.0,86.0,85.0,...,79.0,88.0,68.0,58.0,51.0,15.0,13.0,5.0,10.0,13.0


In [6]:
fifa.dropna(inplace=True)
try:
    fifa.dropna(inplace=True)
except KeyError:
    logger.warning(f"Rows already dropped")

## Questão 1

Qual fração da variância consegue ser explicada pelo primeiro componente principal de `fifa`? Responda como um único float (entre 0 e 1) arredondado para três casas decimais.

In [7]:
def q1():
    # Retorne aqui o resultado da questão 1.
    pca = PCA(n_components=1, random_state=0).fit(fifa)
    evr = pca.explained_variance_ratio_
    return float(evr.round(3))
q1()

0.565

## Questão 2

Quantos componentes principais precisamos para explicar 95% da variância total? Responda como un único escalar inteiro.

In [8]:
pca = PCA(random_state=0).fit(fifa)
evr = pca.explained_variance_ratio_

In [9]:
# g = sns.lineplot(np.arange(len(evr)), np.cumsum(evr))
# g.axes.axhline(0.95, ls="--", color="red")
# plt.xlabel("Number of components"), plt.ylabel("Cumulative explained variance");

In [10]:
np.cumsum(evr)

array([0.56528056, 0.74630577, 0.80583411, 0.84926104, 0.87561427,
       0.88768504, 0.89804009, 0.9065696 , 0.91444823, 0.9216524 ,
       0.9288342 , 0.93516314, 0.94035556, 0.94526355, 0.95000564,
       0.95463336, 0.95888509, 0.96274797, 0.96651386, 0.97026478,
       0.97374294, 0.97701024, 0.98004162, 0.98301578, 0.98558376,
       0.98788729, 0.99003544, 0.9915968 , 0.99315032, 0.99423696,
       0.9953052 , 0.99636412, 0.99732226, 0.99821147, 0.9990712 ,
       0.99980581, 1.        ])

In [11]:
def q2():
    # Retorne aqui o resultado da questão 2.
    cont = 0 + 1
    for n in np.cumsum(evr):
        if n < 0.95:
            cont +=1
    return cont
q2()

15

In [12]:
# ncomp = PCA(n_components=0.95, random_state=0).fit_transform(fifa)
# ncomp.shape

## Questão 3

Qual são as coordenadas (primeiro e segundo componentes principais) do ponto `x` abaixo? O vetor abaixo já está centralizado. Cuidado para __não__ centralizar o vetor novamente (por exemplo, invocando `PCA.transform()` nele). Responda como uma tupla de float arredondados para três casas decimais.

In [31]:
x = [0.87747123,  -1.24990363,  -1.3191255, -36.7341814,
     -35.55091139, -37.29814417, -28.68671182, -30.90902583,
     -42.37100061, -32.17082438, -28.86315326, -22.71193348,
     -38.36945867, -20.61407566, -22.72696734, -25.50360703,
     2.16339005, -27.96657305, -33.46004736,  -5.08943224,
     -30.21994603,   3.68803348, -36.10997302, -30.86899058,
     -22.69827634, -37.95847789, -22.40090313, -30.54859849,
     -26.64827358, -19.28162344, -34.69783578, -34.6614351,
     48.38377664,  47.60840355,  45.76793876,  44.61110193,
     49.28911284
]

In [30]:
def q3():
    # Retorne aqui o resultado da questão 3.
    return tuple(pca.components_.dot(x)[:2].round(3))
q3()

(186.556, -6.592)

## Questão 4

Realiza RFE com estimador de regressão linear para selecionar cinco variáveis, eliminando uma a uma. Quais são as variáveis selecionadas? Responda como uma lista de nomes de variáveis.

In [60]:
def q4():
    # Retorne aqui o resultado da questão 4.
    x = fifa.drop("Overall", axis=1)
    y = fifa["Overall"]
    LR = LinearRegression().fit(x, y)
    rfe = RFE(LR).fit(x,y)
    ndf = pd.DataFrame({'coluna':x.columns,
                    'bool':rfe.get_support(),
                    'coeficientes': pd.Series(LR.coef_)
                     }).sort_values(by='coeficientes', ascending=False).head(5).coluna.to_list()
    return ndf
q4()

['Potential', 'Age', 'Reactions', 'BallControl', 'GKReflexes']